Esse script implementa LSTM para vários arquivos. Todas os arquivos serão colocados em uma variável chamada inpt

Nessa versão o tamanho do ngrama e o modelo forneceram resultados bons. Importante usar data load e tentar melhorar a complexidade do modelo

Esse exemplo tem problema no DataLoader. O objetivo dessa versão é limpar o código e melhorar o modelo.

Essa versão visa fine tunning dos modelos e limpeza do código eliminando as celulas de debug da versão anterior.

Essa versão visa prospectar desempenho com ngram de outros tamanhos

Colocar mais uma camada linear

Essa versão terá a entrada melhorada, sendo coletada para cada arquivo, sem ngrams inter arquivos.

Nessa versão 10 os testes são feitos com a base de dados de validação

Nessa versão 11 as acurácias são apresentadas durante o treinamento

Versão 12 - tentativa de aumentar a complexidade do modelo

Versão 12_1 - tentativa de aumentar a complexidade do modelo da versão 12

Versão 13 adiciona dropout nas camadas LSTM

Versão 14: aumenta a dimensão dos ngrams (quanto maior melhor a acurácia de treinamento) e adiciona regularização L1 e L2.

Versão 15: adiciona visualização da acurácia de validação (early stop não adicionado) 

Versão 16: early stop adicionado

In [1]:
import numpy as np
import random
import torch
#from nltk.tokenize import WordPunctTokenizer
from torch import nn
import os
from tqdm import tqdm
import gc
from torch.utils.data import TensorDataset, DataLoader, random_split
gc.collect()

20

In [2]:
# !pip install numba

In [3]:
# print(nw_model)

In [4]:
# del nw_model

In [5]:
# device_ = get_current_device()
# device.reset()

In [6]:
#Parâmetros
ngram_size = 8#7#6
embed_size = 650#880#1024#800#512#512#700#512 melhor #- 51% #2048 - acuracia de 85% #1024 - acurácia de 85%#128
hidden_size = 650#880#1024#1024#780#700#512  melhor #- 51% #2048 - acuracia de 85% #1024 - acurácia de 85%#128
#melhores (~ 66.5%): (512, 512, 251 epocas), (512, 1024, 125 epocas)
layers_sz = 3#3#4#3#2
# layers_sz = 3#2

# output_size = one_hot_size
dropout_rate = 0.3#0.03#0.3
learning_rate = 0.00004#0.005#0.01
dilMlp = 4
batch_size = 150#1000#3000#3350#950#3950#400#8000#10000#32
# batch_size = 3950#400#8000#10000#32
regularization_type1 = 'L1'
regularization_type2 = 'L2'
lambda_reg=0.000003
lambda_reg2 = 1*lambda_reg
lambda_reg1 = 0.000002 #lambda_reg - lambda_reg2

In [7]:
seed_value = 101

torch.manual_seed(seed_value)

In [8]:
path = 'data/ADFA-LD/Training_Data_Master/'
allFiles = os.listdir(path)
# /content/UTD-0001.txt
# df = pd.read_csv('data/ADFA-LD/TrainingDataMaster/SBUX.csv', index_col = 'Date', parse_dates=True)
print('file: ', allFiles[5])

file:  UTD-0237.txt


In [9]:
len(allFiles)

833

In [10]:
# ngram_size = 6

def create_ngram_sequences(tokens):
  ngrams_list = []
  for i in range(ngram_size, len(tokens) + 1):
    ngrams = tokens[i - ngram_size:i]
    ngrams_list.append(ngrams)
  return ngrams_list

In [11]:
a = [174, 174, 174, 57, 175, 54, 175, 6, 3, 6]
b = create_ngram_sequences(a)
print(b)

[[174, 174, 174, 57, 175, 54, 175, 6], [174, 174, 57, 175, 54, 175, 6, 3], [174, 57, 175, 54, 175, 6, 3, 6]]


In [12]:
inpt = []
ngram_sequences_ = []
for file in tqdm(allFiles):
    # print("File: ", file)
    inpt_ = []
    fileInpt = open(path + file, "r")
    content=fileInpt.readlines()
    inptString = content[0]
    myList = inptString.split()
    corpus_tokens = myList
    for elem in myList:
        inpt = inpt + [int(elem)]
        inpt_ = inpt_ + [int(elem)]
    ngram_sequences_ = ngram_sequences_ + create_ngram_sequences(inpt_)
    ngram_sequences_[:10]
    #len(corpus_tokens)
print(inpt[1:100]) 
    

100%|██████████████████████████████████████████████████████████████████████████████| 833/833 [02:04<00:00,  6.67it/s]

[174, 174, 174, 57, 175, 54, 175, 6, 3, 6, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 11, 45, 33, 192, 33, 5, 197, 192, 6, 33, 5, 3, 197, 192, 192, 6, 33, 5, 3, 197, 192, 192, 6, 33, 5, 3, 197, 192, 192, 192, 6, 33, 5, 3, 197, 192, 192, 192, 6, 33, 5, 3, 197, 192, 192, 6, 33, 5, 3, 197, 192, 192, 192, 6, 33, 5, 3, 197, 192, 192, 6, 192, 192, 243, 125, 125, 125, 125, 125, 125]


In [13]:
ngram_sequences_[:10]
len(ngram_sequences_)

302246

In [14]:
def extract_corpus_vocab(all_words):
  vocab_words = list(set(all_words))
  vocab_words.sort()
  return vocab_words

In [15]:
vocab_words_list = extract_corpus_vocab(inpt)

print(vocab_words_list[0:10])
print(max(vocab_words_list))

[1, 3, 4, 5, 6, 7, 8, 9, 10, 11]
340


In [16]:
vocab_words_list = range(1, max(vocab_words_list) + 1)
len(vocab_words_list)


340

In [17]:
word_to_nums = (vocab_words_list)

In [18]:
ngram_sequences = create_ngram_sequences(inpt)
ngram_sequences[:10]
#len(corpus_tokens)

[[175, 174, 174, 174, 57, 175, 54, 175],
 [174, 174, 174, 57, 175, 54, 175, 6],
 [174, 174, 57, 175, 54, 175, 6, 3],
 [174, 57, 175, 54, 175, 6, 3, 6],
 [57, 175, 54, 175, 6, 3, 6, 174],
 [175, 54, 175, 6, 3, 6, 174, 174],
 [54, 175, 6, 3, 6, 174, 174, 174],
 [175, 6, 3, 6, 174, 174, 174, 174],
 [6, 3, 6, 174, 174, 174, 174, 174],
 [3, 6, 174, 174, 174, 174, 174, 174]]

In [19]:
len(ngram_sequences)

308070

In [20]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# # model.cpu()
# # del  nw_model, checkpoint
# gc.collect()
# torch.cuda.empty_cache()

# print(device)

In [21]:
ngram_seq_nums = ngram_sequences_
# ngram_sequences_to_numbers

In [22]:
ngram_nums = torch.tensor(ngram_seq_nums, device='cpu')
ngram_nums[:10]

tensor([[175, 174, 174, 174,  57, 175,  54, 175],
        [174, 174, 174,  57, 175,  54, 175,   6],
        [174, 174,  57, 175,  54, 175,   6,   3],
        [174,  57, 175,  54, 175,   6,   3,   6],
        [ 57, 175,  54, 175,   6,   3,   6, 174],
        [175,  54, 175,   6,   3,   6, 174, 174],
        [ 54, 175,   6,   3,   6, 174, 174, 174],
        [175,   6,   3,   6, 174, 174, 174, 174],
        [  6,   3,   6, 174, 174, 174, 174, 174],
        [  3,   6, 174, 174, 174, 174, 174, 174]])

In [23]:

X_train = ngram_nums[:, :-1]
y_target = ngram_nums[:, -1]

In [24]:
len(X_train)

302246

In [25]:
y_target[:10]
y_train = y_target

Colocar a parte de dataloader aqui

In [26]:
for idx in y_target[:10]:
  print(idx)

tensor(175)
tensor(6)
tensor(3)
tensor(6)
tensor(174)
tensor(174)
tensor(174)
tensor(174)
tensor(174)
tensor(174)


In [27]:
one_hot_size = len(vocab_words_list)
print(one_hot_size)
print(max(vocab_words_list))
one_hot_size = max(vocab_words_list) +1
print(one_hot_size)
def word_num_to_onehot(target):
  one_hot_list = []
  for idx in target:
    one_hot = np.zeros(one_hot_size, dtype=np.float32)
    one_hot[idx] = 1.0
    one_hot_list.append(one_hot)
  return one_hot_list

340
340
341


In [28]:
device = 'cpu'

# nw_model.to(device_cpu)

In [29]:
y_one_hot = word_num_to_onehot(y_target)
y_one_hot_target_ = np.asarray(y_one_hot)
y_one_hot_target = torch.tensor(np.asarray(y_one_hot), device=device)
# print(y_one_hot_target[0])
# print(y_target[0])

In [30]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model.cpu()
# del  nw_model, checkpoint
gc.collect()
torch.cuda.empty_cache()

print(device)

cuda


In [31]:
num_features = X_train.shape[1]
vocab_size = one_hot_size
# embed_size = 512#512#700#512 melhor #- 51% #2048 - acuracia de 85% #1024 - acurácia de 85%#128
# hidden_size = 1024#780#700#512  melhor #- 51% #2048 - acuracia de 85% #1024 - acurácia de 85%#128
# #melhores (~ 66.5%): (512, 512, 251 epocas), (512, 1024, 125 epocas)
# layers_sz = 2
output_size = one_hot_size
# dropout_rate = 0.03#0.3

class NextWordLSTM(nn.Module):
  def __init__(self, vocab_sz, embed_sz, hidden_sz, output_sz):
    super(NextWordLSTM, self).__init__()
    self.embed = nn.Embedding(vocab_sz, embed_sz)
    self.lstm = nn.LSTM(input_size=embed_sz, hidden_size=hidden_sz, num_layers=layers_sz, dropout=dropout_rate)
    self.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            # nn.Linear(hidden_size*num_features, hidden_size*num_features),
            nn.Linear(hidden_size*num_features, dilMlp*hidden_size*num_features),
            nn.ReLU())
    self.fc1 = nn.Sequential(
            nn.Dropout(dropout_rate),
            # nn.Linear(hidden_size*num_features, output_sz),
            nn.Linear(dilMlp*hidden_size*num_features, hidden_size*num_features),
            nn.ReLU()
            )
    self.fc2 = nn.Sequential(
        nn.Dropout(dropout_rate),
        nn.Linear(hidden_size*num_features, output_sz),
        )
    # self.dropout = nn.Dropout(dropout_rate)
    # self.linear = nn.Linear(hidden_size*num_features, output_sz)

  def forward(self, x_in: torch.Tensor):
    embedded = self.embed(x_in)
    output, _ = self.lstm(embedded)
    output = output.view(output.size(0), -1)
    # output = self.dropout(output)
    # output = self.linear(output)
    output = self.fc(output)
    output = self.fc1(output)
    output = self.fc2(output)
    return output

In [32]:
nw_model = NextWordLSTM(vocab_size, embed_size, hidden_size, output_size)
nw_model.to(device)

NextWordLSTM(
  (embed): Embedding(341, 650)
  (lstm): LSTM(650, 650, num_layers=3, dropout=0.3)
  (fc): Sequential(
    (0): Dropout(p=0.3, inplace=False)
    (1): Linear(in_features=4550, out_features=18200, bias=True)
    (2): ReLU()
  )
  (fc1): Sequential(
    (0): Dropout(p=0.3, inplace=False)
    (1): Linear(in_features=18200, out_features=4550, bias=True)
    (2): ReLU()
  )
  (fc2): Sequential(
    (0): Dropout(p=0.3, inplace=False)
    (1): Linear(in_features=4550, out_features=341, bias=True)
  )
)

In [33]:
criterion = nn.CrossEntropyLoss()
criterion.to(device)

CrossEntropyLoss()

In [34]:
# learning_rate = 0.0005#0.005#0.01

optimizer = torch.optim.Adam(nw_model.parameters(), lr=learning_rate)

In [35]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments: TRUE"
# print(torch.cuda.memory_summary())
# torch.cuda.memory_summary()

In [36]:
print(X_train[0:10])

tensor([[175, 174, 174, 174,  57, 175,  54],
        [174, 174, 174,  57, 175,  54, 175],
        [174, 174,  57, 175,  54, 175,   6],
        [174,  57, 175,  54, 175,   6,   3],
        [ 57, 175,  54, 175,   6,   3,   6],
        [175,  54, 175,   6,   3,   6, 174],
        [ 54, 175,   6,   3,   6, 174, 174],
        [175,   6,   3,   6, 174, 174, 174],
        [  6,   3,   6, 174, 174, 174, 174],
        [  3,   6, 174, 174, 174, 174, 174]])


In [37]:
#Normalization of input
# X_train = X_train/341

In [38]:
import os
from torch.utils.data import TensorDataset, DataLoader, random_split
# data = TensorDataset(X_train_, y_one_hot_target_)
print(X_train.shape)
print(y_one_hot_target.shape)
type(y_one_hot_target)
#torch.tensor
data = TensorDataset(X_train, y_one_hot_target)

torch.Size([302246, 7])
torch.Size([302246, 341])


In [39]:
# train_size = int(0.108 * len(data))
train_size = int(0.8* len(data))
valid_size = len(data) - train_size
# batch_size = 10000#400#8000#10000#32

In [40]:
train_data, valid_data = random_split(data, [train_size, valid_size])

In [41]:
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)

In [42]:
# # # Carrega modelo pre-treinado
# # PATH = 'modelos/512_1024'
# PATH = 'modelos/5_800_800_1024_lr_dropout_3_layers'
# # torch.save(nw_model, PATH)
# nw_model = torch.load(PATH, weights_only=False)
# nw_model.to(device)
# # # model.eval()

In [43]:
class EarlyStopper:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [44]:
early_stopper = EarlyStopper(patience=10, min_delta=0.1) #falta testar patience = 5 e min_delta = 0

In [ ]:
gc.collect
torch.cuda.empty_cache()
num_epochs = 700#751#301#651 #251
len_data = len(train_loader.dataset)
# early_stopper = EarlyStopper(patience=5, min_delta=0) #falta testar patience = 5 e min_delta = 0

for epoch in range(1, num_epochs):
  running_loss = 0
  runningLossValid = 0
  epoch_losses = []
  nw_model.train() #####
  accuracy = 0
  totalSamples = 0
  accuracyValid = 0
  totalSamplesValid = 0

  for batch_X, batch_y in tqdm(train_loader):
      batch_X, batch_y = batch_X.to(device), batch_y.to(device)
      optimizer.zero_grad()
      outputs = nw_model(batch_X)
      loss = criterion(outputs, batch_y.argmax(dim=1)) #batch_y tem shape: batch_size X 341, por isso dim = 1. argmax pega p argmax de 
                                                      #cada linha
      ############### Parte da regularização L1 e L2 #############################
      # Apply L1 regularization
      if regularization_type1 == 'L1':
        l1_norm = sum(p.abs().sum() for p in nw_model.parameters())
        loss += lambda_reg1 * l1_norm
        
      # Apply L2 regularization
      elif regularization_type2 == 'L2':
        l2_norm = sum(p.pow(2).sum() for p in nw_model.parameters())
        loss += lambda_reg2 * l2_norm  

      ##################### Loss normalizado ##################
      running_loss += loss.item() * batch_X.size(0)
      epoch_loss = running_loss / len_data
      epoch_losses.append(epoch_loss)  
      
      ######### parte de cáculo da acurácia ####################
      prediction = outputs.argmax(dim=1) 
      realValue = batch_y.argmax(dim=1)
      accuracy += (prediction == realValue).sum().item()
      totalSamples += realValue.size(0)
    
      
      loss.backward()
      optimizer.step()
      
  accuracy = accuracy/totalSamples

  nw_model.eval() #####
  for batch_X, batch_y in tqdm(valid_loader):
      batch_X, batch_y = batch_X.to(device), batch_y.to(device)
      
      outputs = nw_model(batch_X)
      lossValid = criterion(outputs, batch_y.argmax(dim=1)) #batch_y tem shape: batch_size X 341, por isso dim = 1. argmax pega p argmax de 
                                                      #cada linha
      # ##################### Loss normalizado ##################
      # running_loss += loss.item() * batch_X.size(0)
      # epoch_loss = running_loss / len_data
      # epoch_losses.append(epoch_loss)  
      
      ######### parte de cáculo da acurácia ####################
      predictionValid = outputs.argmax(dim=1) 
      realValueValid = batch_y.argmax(dim=1)
      accuracyValid += (prediction == realValue).sum().item()
      totalSamplesValid += realValue.size(0)

      ################ loss normalizado validação ###############
      runningLossValid += lossValid.item() * batch_X.size(0)
      epochLossValid = runningLossValid / len_data
    


  # if early_stopper.early_stop(lossValid):             
  if epoch > 300:
      if early_stopper.early_stop(epochLossValid):          
          break
  accuracyValid = accuracyValid/totalSamplesValid  
  print(f'Next Word Model LSTM -> Epoch: {epoch}, Loss: {loss}, Train Accuracy: {accuracy}, Validation Accuracy: {accuracyValid}, Validation Loss: {epochLossValid}')
  # if epoch % 10 == 0:
  #   print(f'Next Word Model LSTM -> Epoch: {epoch}, Loss: {loss}')
    
  # if epoch % 25 == 0:
  #   print(f'Next Word Model LSTM -> Epoch: {epoch}, Loss: {loss}')
  

100%|██████████████████████████████████████████████████████████████████████████████| 403/403 [00:49<00:00,  8.10it/s]


Next Word Model LSTM -> Epoch: 1, Loss: 2.2842843532562256, Train Accuracy: 0.6527816837333951, Validation Accuracy: 0.6438356164383562, Validation Loss: 0.28668506838135155


100%|██████████████████████████████████████████████████████████████████████████████| 403/403 [00:49<00:00,  8.16it/s]


Next Word Model LSTM -> Epoch: 2, Loss: 1.8822417259216309, Train Accuracy: 0.6865332759847144, Validation Accuracy: 0.726027397260274, Validation Loss: 0.26565969663785743


100%|██████████████████████████████████████████████████████████████████████████████| 403/403 [00:49<00:00,  8.14it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████| 403/403 [00:49<00:00,  8.08it/s]


Next Word Model LSTM -> Epoch: 238, Loss: 0.9053455591201782, Train Accuracy: 0.8760070472629821, Validation Accuracy: 0.8493150684931506, Validation Loss: 0.25206762804241817


100%|██████████████████████████████████████████████████████████████████████████████| 403/403 [00:49<00:00,  8.08it/s]


Next Word Model LSTM -> Epoch: 239, Loss: 0.6875624656677246, Train Accuracy: 0.8760649473109563, Validation Accuracy: 0.8835616438356164, Validation Loss: 0.2535585641336393


  9%|███████                                                                      | 149/1612 [01:01<10:03,  2.42it/s]

In [ ]:
early_stopper.early_stop(lossValid)

In [ ]:
early_stopper.counter

In [ ]:
early_stopper.min_validation_loss

In [ ]:
# a = batch_y.argmax(dim=1)
# b0 = outputs
# b = outputs.argmax(dim=1)
# c = a==b
# total_correct = c.sum().item()
# # print(total_correct)
# a.size(0)
# # b0[0,:].size()

In [ ]:
# PATH = 'modelos/512_1024'
# model = torch.load(PATH, weights_only=False)
# model.eval()
model = nw_model

In [ ]:
# PATH = 'modelos/5_512_1024_lr_dropout_3_layers'
# torch.save(nw_model, PATH)

In [ ]:
device_cpu = 'cpu'

# nw_model.to(device_cpu)
model.to(device_cpu)
model.eval()

In [ ]:
def pick_ngram_sub_sequence(seq_list):
  idx = random.randint(0, len(seq_list))
  return seq_list[idx], seq_list[idx][:-1]

In [ ]:
num_trials = 10
acertos = 0
for i in range(1, num_trials+1):
  ngram_seq_pick, ngram_sub_seq = pick_ngram_sub_sequence(ngram_sequences)
  print(f'Trial #{i} - Picked sequence: {ngram_seq_pick}, Test sub-sequence: {ngram_sub_seq}')
  # ngram_sub_seq_num = ngram_sequences_to_numbers([ngram_sub_seq])
  ngram_sub_seq_num = [ngram_sub_seq]
  X_test = torch.tensor(ngram_sub_seq_num)
  print(f'Trial #{i} - X_test: {X_test}')
  model.eval()
  with torch.no_grad():
    # y_predict = nw_model(X_test)
    y_predict = model(X_test)
    idx = torch.argmax(y_predict)
    # print('idx: ', idx)
    print(f'Trial #{i} - idx: {idx}, next word: {vocab_words_list[idx-1]}')
    print(f'Trial #{i} - Real next word: {ngram_seq_pick[-1]}')
    
    if vocab_words_list[idx-1] == int(ngram_seq_pick[-1]):
        acertos = acertos + 1
    # print(acertos)
print('percentual de acertos: ', 100*acertos/num_trials)     

In [ ]:
PATH = 'modelos/7_650_3x650_3_linear_layers_4550_18200_4550_341_L1_L2_EarlyStop300_totalData'

torch.save(nw_model, PATH)

In [63]:
num_trials = int(0.05*len(ngram_sequences))
# num_trials = int(0.1*len(ngram_sequences_valid))
acertos = 0
for i in tqdm(range(num_trials)):
  # ngram_seq_pick, ngram_sub_seq = pick_ngram_sub_sequence(ngram_sequences)
  # print(type(i))
  ngram_sub_seq = ngram_sequences[i][:-1]
  ngram_seq_pick= ngram_sequences[i][:]
  # print(ngram_sequences[i][:])
  # print(ngram_seq_pick)
  # print(ngram_sub_seq)
    
  # print(f'Trial #{i} - Picked sequence: {ngram_seq_pick}, Test sub-sequence: {ngram_sub_seq}')
  ngram_sub_seq_num = [ngram_sub_seq]
  # print(ngram_sub_seq_num)
  X_test = torch.tensor(ngram_sub_seq_num)
  # print(f'Trial #{i} - X_test: {X_test}')
  model.eval()
  with torch.no_grad():
    # y_predict = nw_model(X_test)
    y_predict = model(X_test)
    idx = torch.argmax(y_predict)
    # print('idx: ', idx)
    # print(f'Trial #{i} - idx: {idx}, next word: {vocab_words_list[idx-1]}')
    # print(f'Trial #{i} - Real next word: {ngram_seq_pick[-1]}')
    
    if vocab_words_list[idx-1] == int(ngram_seq_pick[-1]):
        acertos = acertos + 1
    # print(acertos)
print('percentual de acertos: ', 100*acertos/num_trials)

  1%|▊                                                                           | 154/15403 [00:13<22:26, 11.32it/s]


KeyboardInterrupt: 

In [46]:
# PATH = 'modelos/5_800_800_800_1024_linear_layers_5120_5120_341'
# torch.save(nw_model, PATH)

################################################################################################################################
####################### Validação com dados não vistos ##########################################################################
#################################################################################################################################

Percentual de acertos para dados de validação: 61,5%. Muito similar ao nível de erro do conjunto de dados de treinamento.